Based on https://github.com/samwit/langchain-tutorials

Before running, create a new virtual environment

```
py -m venv .venv
```

Then activate it

In Bash:
```
source .venv/bin/activate
```

CMD:
```
.venv\Scripts\activate
```

LangChain is a framework available in Python and Node. It provides a lot of tools useful for building simple (and somewhat advanced) applications. Access to several models, conversation templates and tools. It uses "chains" which combine LLMs, formatted prompts, memory and tools to create useful applications.

Here, we will cover 2 core topics:
- Conversations with Memory
- Tools

After this, you should have the skills necessary to build a simple application.

In [ ]:
%pip install -r requirements.txt

In [10]:
import  os
from dotenv import load_dotenv
load_dotenv()

True

### Chat with memory

In [8]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferMemory

In [11]:
llm = ChatOpenAI(
    openai_api_key = os.getenv("OPENAI_API_KEY"),
    model_name = 'gpt-3.5-turbo-0125',
    temperature = 0.3,
)

In [10]:
memory = ConversationBufferMemory()

conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=memory,
)



In [ ]:
# "Hi! My name is {name}"
conversation.predict(input=)

In [ ]:
# "What is my name?"
conversation.predict(input=)

In [ ]:
# "What was my last message?"
conversation.predict(input=)

In [16]:
from langchain.chains.conversation.memory import ConversationSummaryMemory

In [64]:
summary_memory = ConversationSummaryMemory(llm=ChatOpenAI())


In [65]:
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=summary_memory
)


In [ ]:
conversation.predict(input="Hello! Im Artur")

In [ ]:
conversation.predict(input="What is my name?")

In [ ]:
conversation.predict(input="What was my last message?")

In [ ]:
conversation.predict(input="Here are 3 facts about me. I was a student at SFU, I am a software developer, and I like fishing")

In [ ]:
print(conversation.memory.buffer)

## Other approaches

- ConversationBufferWindowMemory: keep last `n` messages, discard the rest
- ConversationSummaryBufferMemory: keep last `n` messages, summarize the rest
- ConversationKGMemory: knowledge graph
- ConversationEntityMemory: extracts key information from the conversation

## Why use different types of memory?
- Save space - keeping all messages can quickly fill up the context window
- Better accuracy - Summarizing or extracting entities can keep the prompts small while retaining all key details


### Tools

Tools are a key component of building non-trivial generative AI applications. The models are inherently limited in the information they have access to.

In [ ]:
# A simple example is asking the model something about the present day.
summary_memory = ConversationSummaryMemory(llm=ChatOpenAI())

conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=summary_memory
)
conversation.predict(input=)

In [ ]:
conversation.predict(input=)

In [ ]:

conversation.predict(input=)

In [3]:
# Even though it claims to check the price, it cannot actually do it. We need to give it that ability explicitly
# Lets give it some tools

from langchain.utilities import WikipediaAPIWrapper
from langchain.tools import DuckDuckGoSearchRun
from langchain.utilities import PythonREPL
from langchain.agents import Tool

wikipedia = WikipediaAPIWrapper()
search = DuckDuckGoSearchRun()
py_repl = PythonREPL()



In [4]:
wikipedia.run("JavaScript")


"Page: JavaScript\nSummary: JavaScript (), often abbreviated as JS, is a programming language and core technology of the World Wide Web, alongside HTML and CSS. As of 2024, 98.9% of websites use JavaScript on the client side for webpage behavior, often incorporating third-party libraries. All major web browsers have a dedicated JavaScript engine to execute the code on users' devices.\nJavaScript is a high-level, often just-in-time compiled language that conforms to the ECMAScript standard. It has dynamic typing, prototype-based object-orientation, and first-class functions. It is multi-paradigm, supporting event-driven, functional, and imperative programming styles. It has application programming interfaces (APIs) for working with text, dates, regular expressions, standard data structures, and the Document Object Model (DOM).\nThe ECMAScript standard does not include any input/output (I/O), such as networking, storage, or graphics facilities. In practice, the web browser or other runti

In [ ]:
search.run("Tesla stock price")


In [ ]:
py_repl.run("def add(a, b):\n    return a + b\n\nprint(add(1, 2))")



In [5]:
tools = [
    Tool(
        name="wikipedia",
        func=wikipedia.run,
        description="Useful when you need to find information about a topic, country or person on wikipedia"
    ),
        Tool(
        name="duckduckgo",
        func=search.run,
        description="Useful for when you need to do a search on the internet to find information that another tool can't find."
    ),
        Tool(
        name="python repl",
        func=py_repl.run,
        description="Useful for when you need to use python to answer a question. You should input python code"
    )
]


In [53]:
print(tools)

[Tool(name='wikipedia', description='Useful when you need to find information about a topic, country or person on wikipedia', func=<bound method WikipediaAPIWrapper.run of WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\artur\\OneDrive - sfu.ca\\Desktop\\Programming\\genai-tutorials\\.venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=3, lang='en', load_all_available_meta=False, doc_content_chars_max=4000)>), Tool(name='duckduckgo', description="Useful for when you need to do a search on the internet to find information that another tool can't find.", func=<bound method BaseTool.run of DuckDuckGoSearchRun()>), Tool(name='python repl', description='Useful for when you need to use python to answer a question. You should input python code', func=<bound method PythonREPL.run of PythonREPL(globals={}, locals={})>)]


In [ ]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    llm=llm,
    tools=tools,
    verbose=True,
)

zero_shot_agent.run("What is 9 + 10")

In [ ]:
zero_shot_agent.run("is 11 a prime number")

In [ ]:
# what does the prompt look like
print(zero_shot_agent.agent.llm_chain.prompt.template)

In [ ]:
zero_shot_agent.run("What is 2 * the price of tesla stock")

## Next steps:

Explore the [Complete list of langchain tools](https://python.langchain.com/docs/integrations/tools/).

Try [custom prompts](https://python.langchain.com/docs/modules/model_io/prompts/)

Build more complex [agents](https://python.langchain.com/docs/modules/agents/)

## Apply what you learned!!

# Hackathon next week

- In small groups
- Think of a problem you face that can be solved by generative AI
- Build an app to solve it
    - Use any model but gpt-3.5 is probably the best starting point
    - Play around with prompting
    - Use at least one external tool (see list above) to enhance the functionality
    - UI doesnt matter but if you have time make it look nice (running in terminal is fine)
    - Don't have to use LangChain if you want to do it from scratch, but this will take longer
    - If you're not comfortable with Python, LangChain is also available for [JavaScript/TypeScript](https://js.langchain.com/docs/get_started). Alternatively, use any language you want with 3rd party libraries or directly calling an LLM API.
    - Winner gets a prize!

We will have demos during the Friday session and choose the best app.
